## 1️⃣ Configuración y Conexión

⚠️ **IMPORTANTE:** Asegúrate de tener SAP2000 abierto antes de ejecutar esta celda.

In [ ]:
# === CELDA DE CONEXIÓN (ejecutar una sola vez) ===
import comtypes.client

# Conectar a instancia activa de SAP2000
try:
    SapObject = comtypes.client.GetActiveObject("CSI.SAP2000.API.SapObject")
    SapModel = SapObject.SapModel
    print("✅ Conectado a SAP2000")
    print(f"📁 Archivo: {SapModel.GetModelFilename()}")
except Exception as e:
    print(f"❌ Error de conexión: {e}")
    print("   Asegúrate de que SAP2000 esté abierto.")

## 2️⃣ Funciones de Utilidad

Helpers para facilitar el trabajo con la API.

In [ ]:
# === FUNCIONES DE UTILIDAD ===

def limpiar_modelo():
    """Crea un modelo nuevo en blanco"""
    ret = SapModel.InitializeNewModel()
    ret2 = SapModel.File.NewBlank()
    if ret2 == 0:
        print("🧹 Modelo limpiado - nuevo modelo en blanco")
    return ret2

def refrescar_vista():
    """Refresca la vista de SAP2000"""
    ret = SapModel.View.RefreshView(0, False)
    print("🔄 Vista refrescada")
    return ret

def estado_modelo():
    """Muestra resumen del modelo actual"""
    pts = SapModel.PointObj.Count()
    frames = SapModel.FrameObj.Count()
    areas = SapModel.AreaObj.Count()
    print("📊 Estado del Modelo:")
    print(f"   Puntos: {pts}")
    print(f"   Frames: {frames}")
    print(f"   Areas: {areas}")
    return {'puntos': pts, 'frames': frames, 'areas': areas}

def set_unidades(unidad='kN_m'):
    """
    Establece las unidades del modelo
    Opciones: 'kN_m', 'kN_mm', 'kgf_m', 'tonf_m', 'kip_ft', 'kip_in'
    """
    unidades = {
        'kN_m': 6,      # kN, m, C
        'kN_mm': 7,     # kN, mm, C
        'kgf_m': 10,    # kgf, m, C
        'tonf_m': 12,   # tonf, m, C
        'kip_ft': 4,    # kip, ft, F
        'kip_in': 3,    # kip, in, F
    }
    ret = SapModel.SetPresentUnits(unidades.get(unidad, 6))
    if ret == 0:
        print(f"📐 Unidades establecidas: {unidad}")
    return ret

print("✅ Funciones de utilidad cargadas")

---
## 3️⃣ Sandbox - Pruebas Individuales

Usa esta sección para probar funciones una por una antes de integrarlas.

### 3.1 Crear Puntos

In [ ]:
# === PRUEBA: Crear punto por coordenadas ===
# API: SapModel.PointObj.AddCartesian(x, y, z, Name, UserName)
# Retorna: [NombreAsignado, RetCode]

limpiar_modelo()
set_unidades('kN_m')

# Crear un punto
x, y, z = 0, 0, 0
ret = SapModel.PointObj.AddCartesian(x, y, z, "", "P1")

print(f"Retorno completo: {ret}")
print(f"Nombre asignado: {ret[0]}")
print(f"RetCode: {ret[-1]} ({'✅ OK' if ret[-1] == 0 else '❌ Error'})")

refrescar_vista()

In [ ]:
# === FUNCIÓN PROBADA: crear_punto ===

def crear_punto(x, y, z, nombre=""):
    """
    Crea un punto en coordenadas cartesianas.
    
    Parámetros:
        x, y, z: coordenadas del punto
        nombre: nombre opcional del punto
    
    Retorna:
        str: nombre asignado al punto, o None si hay error
    """
    ret = SapModel.PointObj.AddCartesian(x, y, z, "", nombre)
    if ret[-1] == 0:
        return ret[0]
    else:
        print(f"❌ Error creando punto: código {ret[-1]}")
        return None

# Probar
limpiar_modelo()
p1 = crear_punto(0, 0, 0, "Base1")
p2 = crear_punto(5, 0, 0, "Base2")
p3 = crear_punto(0, 0, 3, "Top1")
p4 = crear_punto(5, 0, 3, "Top2")

print(f"\nPuntos creados: {p1}, {p2}, {p3}, {p4}")
refrescar_vista()
estado_modelo()

### 3.2 Crear Frames (Elementos Lineales)

In [ ]:
# === PRUEBA: Crear frame por puntos ===
# API: SapModel.FrameObj.AddByPoint(Point1, Point2, Name, PropName, UserName)
# Retorna: [NombreAsignado, RetCode]

# Usar puntos existentes de la celda anterior
ret = SapModel.FrameObj.AddByPoint("Base1", "Top1", "", "Default", "COL1")

print(f"Retorno completo: {ret}")
print(f"Nombre asignado: {ret[0]}")
print(f"RetCode: {ret[-1]} ({'✅ OK' if ret[-1] == 0 else '❌ Error'})")

refrescar_vista()

In [ ]:
# === PRUEBA: Crear frame por coordenadas ===
# API: SapModel.FrameObj.AddByCoord(xi, yi, zi, xj, yj, zj, Name, PropName, UserName)
# Retorna: [NombreAsignado, RetCode]

ret = SapModel.FrameObj.AddByCoord(0, 0, 3, 5, 0, 3, "", "Default", "VIGA1")

print(f"Retorno completo: {ret}")
print(f"Nombre asignado: {ret[0]}")
print(f"RetCode: {ret[-1]} ({'✅ OK' if ret[-1] == 0 else '❌ Error'})")

refrescar_vista()
estado_modelo()

In [ ]:
# === FUNCIONES PROBADAS: crear_frame ===

def crear_frame_por_puntos(punto_i, punto_j, seccion="Default", nombre=""):
    """
    Crea un frame conectando dos puntos existentes.
    
    Parámetros:
        punto_i: nombre del punto inicial
        punto_j: nombre del punto final
        seccion: nombre de la sección a asignar
        nombre: nombre opcional del frame
    
    Retorna:
        str: nombre asignado al frame
    """
    ret = SapModel.FrameObj.AddByPoint(punto_i, punto_j, "", seccion, nombre)
    if ret[-1] == 0:
        return ret[0]
    else:
        print(f"❌ Error creando frame: código {ret[-1]}")
        return None

def crear_frame_por_coord(xi, yi, zi, xj, yj, zj, seccion="Default", nombre=""):
    """
    Crea un frame definiendo coordenadas de inicio y fin.
    
    Parámetros:
        xi, yi, zi: coordenadas del punto inicial
        xj, yj, zj: coordenadas del punto final
        seccion: nombre de la sección a asignar
        nombre: nombre opcional del frame
    
    Retorna:
        str: nombre asignado al frame
    """
    ret = SapModel.FrameObj.AddByCoord(xi, yi, zi, xj, yj, zj, "", seccion, nombre)
    if ret[-1] == 0:
        return ret[0]
    else:
        print(f"❌ Error creando frame: código {ret[-1]}")
        return None

print("✅ Funciones de frames cargadas")

### 3.3 Materiales

In [ ]:
# === PRUEBA: Definir material ===
# API: SapModel.PropMaterial.SetMaterial(Name, MatType)
# MatType: 1=Steel, 2=Concrete, 6=Aluminum, etc.

# Crear material de concreto
ret = SapModel.PropMaterial.SetMaterial("CONC_H30", 2)  # 2 = Concrete
print(f"SetMaterial retorno: {ret}")

# Asignar propiedades isotrópicas
# API: SetMPIsotropic(Name, E, Poisson, ThermalCoeff)
E = 25000000  # kN/m2 (25 GPa)
poisson = 0.2
thermal = 0.00001
ret = SapModel.PropMaterial.SetMPIsotropic("CONC_H30", E, poisson, thermal)
print(f"SetMPIsotropic retorno: {ret}")

In [ ]:
# === FUNCIÓN PROBADA: crear_material ===

def crear_material_concreto(nombre, E_kN_m2, poisson=0.2, thermal=0.00001):
    """
    Crea un material de concreto isotrópico.
    
    Parámetros:
        nombre: nombre del material
        E_kN_m2: módulo de elasticidad en kN/m2
        poisson: coeficiente de Poisson (default 0.2)
        thermal: coef. dilatación térmica (default 1e-5)
    
    Retorna:
        str: nombre del material si éxito, None si error
    """
    ret1 = SapModel.PropMaterial.SetMaterial(nombre, 2)  # 2 = Concrete
    if ret1 != 0:
        print(f"❌ Error en SetMaterial: {ret1}")
        return None
    
    ret2 = SapModel.PropMaterial.SetMPIsotropic(nombre, E_kN_m2, poisson, thermal)
    if ret2 != 0:
        print(f"❌ Error en SetMPIsotropic: {ret2}")
        return None
    
    print(f"✅ Material '{nombre}' creado (E={E_kN_m2/1e6:.1f} GPa)")
    return nombre

def crear_material_acero(nombre, E_kN_m2=200000000, poisson=0.3, thermal=0.0000117):
    """
    Crea un material de acero isotrópico.
    
    Parámetros:
        nombre: nombre del material
        E_kN_m2: módulo de elasticidad en kN/m2 (default 200 GPa)
        poisson: coeficiente de Poisson (default 0.3)
        thermal: coef. dilatación térmica (default 1.17e-5)
    
    Retorna:
        str: nombre del material si éxito, None si error
    """
    ret1 = SapModel.PropMaterial.SetMaterial(nombre, 1)  # 1 = Steel
    if ret1 != 0:
        print(f"❌ Error en SetMaterial: {ret1}")
        return None
    
    ret2 = SapModel.PropMaterial.SetMPIsotropic(nombre, E_kN_m2, poisson, thermal)
    if ret2 != 0:
        print(f"❌ Error en SetMPIsotropic: {ret2}")
        return None
    
    print(f"✅ Material '{nombre}' creado (E={E_kN_m2/1e6:.1f} GPa)")
    return nombre

# Probar
limpiar_modelo()
crear_material_concreto("H30", 25e6)
crear_material_acero("A36")

### 3.4 Secciones de Frame

In [ ]:
# === PRUEBA: Crear sección rectangular ===
# API: SapModel.PropFrame.SetRectangle(Name, MatProp, Depth, Width)

ret = SapModel.PropFrame.SetRectangle("COL40x40", "H30", 0.4, 0.4)
print(f"SetRectangle retorno: {ret}")

# Crear sección para viga
ret = SapModel.PropFrame.SetRectangle("VIGA30x50", "H30", 0.5, 0.3)
print(f"SetRectangle retorno: {ret}")

In [ ]:
# === FUNCIÓN PROBADA: crear_seccion ===

def crear_seccion_rectangular(nombre, material, alto, ancho):
    """
    Crea una sección rectangular para frames.
    
    Parámetros:
        nombre: nombre de la sección
        material: nombre del material (debe existir)
        alto: altura de la sección (depth)
        ancho: ancho de la sección (width)
    
    Retorna:
        str: nombre de la sección si éxito
    """
    ret = SapModel.PropFrame.SetRectangle(nombre, material, alto, ancho)
    if ret == 0:
        print(f"✅ Sección '{nombre}' creada ({ancho}x{alto})")
        return nombre
    else:
        print(f"❌ Error creando sección: {ret}")
        return None

def crear_seccion_circular(nombre, material, diametro):
    """
    Crea una sección circular para frames.
    
    Parámetros:
        nombre: nombre de la sección
        material: nombre del material (debe existir)
        diametro: diámetro de la sección
    
    Retorna:
        str: nombre de la sección si éxito
    """
    ret = SapModel.PropFrame.SetCircle(nombre, material, diametro)
    if ret == 0:
        print(f"✅ Sección '{nombre}' creada (Ø{diametro})")
        return nombre
    else:
        print(f"❌ Error creando sección: {ret}")
        return None

# Probar
crear_seccion_rectangular("COL50x50", "H30", 0.5, 0.5)
crear_seccion_rectangular("VIGA40x60", "H30", 0.6, 0.4)
crear_seccion_circular("PILAR_D50", "H30", 0.5)

### 3.5 Restricciones (Apoyos)

In [ ]:
# === PRUEBA: Asignar apoyo empotrado ===
# API: SapModel.PointObj.SetRestraint(Name, Value[])
# Value es array de 6 booleanos: [UX, UY, UZ, RX, RY, RZ]

# Primero creamos un modelo simple
limpiar_modelo()
set_unidades('kN_m')
crear_material_concreto("H30", 25e6)
crear_seccion_rectangular("COL40x40", "H30", 0.4, 0.4)

# Crear columna
frame = crear_frame_por_coord(0, 0, 0, 0, 0, 3, "COL40x40", "COL1")

# Obtener puntos del frame
ret = SapModel.FrameObj.GetPoints(frame, "", "")
print(f"GetPoints retorno: {ret}")
punto_i = ret[0]  # punto inicial
punto_j = ret[1]  # punto final
print(f"Punto I (base): {punto_i}")
print(f"Punto J (top): {punto_j}")

# Empotrar base (todos restringidos)
empotrado = [True, True, True, True, True, True]
ret = SapModel.PointObj.SetRestraint(punto_i, empotrado)
print(f"SetRestraint retorno: {ret}")

refrescar_vista()

In [ ]:
# === FUNCIONES PROBADAS: restricciones ===

def empotrar_punto(nombre_punto):
    """Empotra un punto (restringe todos los GDL)"""
    restriccion = [True, True, True, True, True, True]
    ret = SapModel.PointObj.SetRestraint(nombre_punto, restriccion)
    if ret == 0:
        print(f"✅ Punto '{nombre_punto}' empotrado")
    return ret

def articular_punto(nombre_punto):
    """Articula un punto (solo traslaciones restringidas)"""
    restriccion = [True, True, True, False, False, False]
    ret = SapModel.PointObj.SetRestraint(nombre_punto, restriccion)
    if ret == 0:
        print(f"✅ Punto '{nombre_punto}' articulado")
    return ret

def apoyo_simple_punto(nombre_punto):
    """Apoyo simple - solo UZ restringido (apoyo vertical)"""
    restriccion = [False, False, True, False, False, False]
    ret = SapModel.PointObj.SetRestraint(nombre_punto, restriccion)
    if ret == 0:
        print(f"✅ Punto '{nombre_punto}' con apoyo simple")
    return ret

def restriccion_personalizada(nombre_punto, ux=False, uy=False, uz=False, rx=False, ry=False, rz=False):
    """Aplica restricción personalizada"""
    restriccion = [ux, uy, uz, rx, ry, rz]
    ret = SapModel.PointObj.SetRestraint(nombre_punto, restriccion)
    if ret == 0:
        print(f"✅ Restricción aplicada a '{nombre_punto}'")
    return ret

print("✅ Funciones de restricciones cargadas")

### 3.6 Patrones de Carga

In [ ]:
# === PRUEBA: Agregar patrones de carga ===
# API: SapModel.LoadPatterns.Add(Name, MyType, SelfWTMultiplier)
# MyType: 1=Dead, 2=SuperDead, 3=Live, 4=ReduceLive, 5=Quake, etc.

# El patrón DEAD ya existe por defecto, agregamos otros
ret = SapModel.LoadPatterns.Add("LIVE", 3, 0)  # 3 = Live
print(f"Agregar LIVE: {ret}")

ret = SapModel.LoadPatterns.Add("SDL", 2, 0)   # 2 = SuperDead
print(f"Agregar SDL: {ret}")

ret = SapModel.LoadPatterns.Add("SISMO_X", 5, 0)  # 5 = Quake
print(f"Agregar SISMO_X: {ret}")

In [ ]:
# === FUNCIÓN PROBADA: patrones de carga ===

def agregar_patron_carga(nombre, tipo='dead', factor_pp=0):
    """
    Agrega un patrón de carga al modelo.
    
    Parámetros:
        nombre: nombre del patrón
        tipo: 'dead', 'live', 'superdead', 'quake', 'wind', 'snow', 'other'
        factor_pp: multiplicador de peso propio
    
    Retorna:
        int: 0 si éxito
    """
    tipos = {
        'dead': 1,
        'superdead': 2,
        'live': 3,
        'reducelive': 4,
        'quake': 5,
        'wind': 6,
        'snow': 7,
        'other': 8
    }
    tipo_num = tipos.get(tipo.lower(), 8)
    ret = SapModel.LoadPatterns.Add(nombre, tipo_num, factor_pp)
    if ret == 0:
        print(f"✅ Patrón '{nombre}' agregado (tipo: {tipo})")
    else:
        print(f"⚠️ Patrón '{nombre}': código {ret}")
    return ret

# Probar
agregar_patron_carga("CV", "live")
agregar_patron_carga("CM", "superdead")
agregar_patron_carga("VIENTO", "wind")

### 3.7 Cargas en Elementos

In [ ]:
# === PRUEBA: Carga distribuida en frame ===
# API: SetLoadDistributed(Name, LoadPat, MyType, Dir, Dist1, Dist2, Val1, Val2, CSys, RelDist, Replace)

# Crear modelo de prueba
limpiar_modelo()
set_unidades('kN_m')
crear_material_concreto("H30", 25e6)
crear_seccion_rectangular("VIGA30x50", "H30", 0.5, 0.3)

# Crear viga simplemente apoyada
viga = crear_frame_por_coord(0, 0, 0, 6, 0, 0, "VIGA30x50", "V1")

# Obtener puntos y restringir
ret = SapModel.FrameObj.GetPoints(viga, "", "")
articular_punto(ret[0])  # apoyo izquierdo
apoyo_simple_punto(ret[1])  # apoyo derecho (deslizante)

# Agregar carga distribuida (10 kN/m hacia abajo en eje Z global)
# Dir: 1=Local1, 2=Local2, 3=Local3, 4=X, 5=Y, 6=Z, 7=Gravity, etc.
ret = SapModel.FrameObj.SetLoadDistributed(viga, "DEAD", 1, 6, 0, 1, -10, -10, "Global", True, True)
print(f"SetLoadDistributed retorno: {ret}")

refrescar_vista()

In [ ]:
# === FUNCIÓN PROBADA: cargas ===

def carga_distribuida_frame(frame, patron, w_kN_m, direccion='gravity'):
    """
    Aplica carga uniformemente distribuida a un frame.
    
    Parámetros:
        frame: nombre del frame
        patron: nombre del patrón de carga
        w_kN_m: carga en kN/m (negativo hacia abajo)
        direccion: 'gravity', 'x', 'y', 'z', 'local1', 'local2', 'local3'
    
    Retorna:
        int: 0 si éxito
    """
    dirs = {'local1': 1, 'local2': 2, 'local3': 3, 'x': 4, 'y': 5, 'z': 6, 
            'gravity': 7, 'projectedx': 8, 'projectedy': 9, 'projectedz': 10}
    dir_num = dirs.get(direccion.lower(), 7)
    
    ret = SapModel.FrameObj.SetLoadDistributed(
        frame, patron, 1, dir_num, 0, 1, w_kN_m, w_kN_m, "Global", True, True
    )
    if ret == 0:
        print(f"✅ Carga {w_kN_m} kN/m aplicada a '{frame}'")
    return ret

def carga_puntual_frame(frame, patron, p_kN, posicion_relativa=0.5, direccion='gravity'):
    """
    Aplica carga puntual a un frame.
    
    Parámetros:
        frame: nombre del frame
        patron: nombre del patrón de carga
        p_kN: carga en kN (negativo hacia abajo)
        posicion_relativa: 0 a 1, posición de la carga
        direccion: 'gravity', 'x', 'y', 'z', 'local1', 'local2', 'local3'
    """
    dirs = {'local1': 1, 'local2': 2, 'local3': 3, 'x': 4, 'y': 5, 'z': 6, 
            'gravity': 7, 'projectedx': 8, 'projectedy': 9, 'projectedz': 10}
    dir_num = dirs.get(direccion.lower(), 7)
    
    ret = SapModel.FrameObj.SetLoadPoint(
        frame, patron, 1, dir_num, posicion_relativa, p_kN, "Global", True, True
    )
    if ret == 0:
        print(f"✅ Carga {p_kN} kN aplicada a '{frame}' en pos={posicion_relativa}")
    return ret

# Probar
carga_distribuida_frame(viga, "DEAD", -15)  # 15 kN/m hacia abajo

### 3.8 Análisis

In [ ]:
# === PRUEBA: Correr análisis ===
# Primero guardar el modelo (requerido antes de analizar)

import os

# Guardar modelo
ruta_modelo = os.path.join(os.getcwd(), "modelo_test.sdb")
ret = SapModel.File.Save(ruta_modelo)
print(f"Guardar modelo: {ret}")

# Correr análisis
ret = SapModel.Analyze.RunAnalysis()
print(f"RunAnalysis retorno: {ret}")

if ret == 0:
    print("✅ Análisis completado")
else:
    print("❌ Error en análisis")

In [ ]:
# === FUNCIÓN PROBADA: análisis ===

def guardar_modelo(ruta=None):
    """Guarda el modelo actual"""
    if ruta is None:
        ruta = os.path.join(os.getcwd(), "modelo_sap.sdb")
    ret = SapModel.File.Save(ruta)
    if ret == 0:
        print(f"💾 Modelo guardado: {ruta}")
    return ret

def correr_analisis(guardar=True):
    """Corre el análisis del modelo"""
    if guardar:
        guardar_modelo()
    
    print("⏳ Ejecutando análisis...")
    ret = SapModel.Analyze.RunAnalysis()
    if ret == 0:
        print("✅ Análisis completado exitosamente")
    else:
        print(f"❌ Error en análisis: {ret}")
    return ret

print("✅ Funciones de análisis cargadas")

---
## 4️⃣ Consolidación - Todas las Funciones Probadas

Esta celda contiene todas las funciones validadas listas para usar.

In [ ]:
# === MÓDULO CONSOLIDADO DE FUNCIONES PROBADAS ===

class SAP2000Helper:
    """
    Clase helper con funciones probadas para SAP2000 API.
    Todas las funciones manejan correctamente los retornos de comtypes.
    """
    
    def __init__(self, sap_model):
        self.SapModel = sap_model
    
    # === UTILIDADES ===
    def limpiar(self):
        self.SapModel.InitializeNewModel()
        return self.SapModel.File.NewBlank()
    
    def refrescar(self):
        return self.SapModel.View.RefreshView(0, False)
    
    def set_unidades(self, unidad='kN_m'):
        unidades = {'kN_m': 6, 'kN_mm': 7, 'kgf_m': 10, 'tonf_m': 12, 'kip_ft': 4, 'kip_in': 3}
        return self.SapModel.SetPresentUnits(unidades.get(unidad, 6))
    
    def estado(self):
        return {
            'puntos': self.SapModel.PointObj.Count(),
            'frames': self.SapModel.FrameObj.Count(),
            'areas': self.SapModel.AreaObj.Count()
        }
    
    # === GEOMETRÍA ===
    def punto(self, x, y, z, nombre=""):
        ret = self.SapModel.PointObj.AddCartesian(x, y, z, "", nombre)
        return ret[0] if ret[-1] == 0 else None
    
    def frame_por_puntos(self, pi, pj, seccion="Default", nombre=""):
        ret = self.SapModel.FrameObj.AddByPoint(pi, pj, "", seccion, nombre)
        return ret[0] if ret[-1] == 0 else None
    
    def frame_por_coord(self, xi, yi, zi, xj, yj, zj, seccion="Default", nombre=""):
        ret = self.SapModel.FrameObj.AddByCoord(xi, yi, zi, xj, yj, zj, "", seccion, nombre)
        return ret[0] if ret[-1] == 0 else None
    
    def puntos_de_frame(self, frame):
        ret = self.SapModel.FrameObj.GetPoints(frame, "", "")
        return (ret[0], ret[1]) if ret[-1] == 0 else (None, None)
    
    # === MATERIALES ===
    def material_concreto(self, nombre, E_kN_m2, poisson=0.2):
        ret1 = self.SapModel.PropMaterial.SetMaterial(nombre, 2)
        ret2 = self.SapModel.PropMaterial.SetMPIsotropic(nombre, E_kN_m2, poisson, 1e-5)
        return nombre if ret1 == 0 and ret2 == 0 else None
    
    def material_acero(self, nombre, E_kN_m2=200e6, poisson=0.3):
        ret1 = self.SapModel.PropMaterial.SetMaterial(nombre, 1)
        ret2 = self.SapModel.PropMaterial.SetMPIsotropic(nombre, E_kN_m2, poisson, 1.17e-5)
        return nombre if ret1 == 0 and ret2 == 0 else None
    
    # === SECCIONES ===
    def seccion_rectangular(self, nombre, material, alto, ancho):
        ret = self.SapModel.PropFrame.SetRectangle(nombre, material, alto, ancho)
        return nombre if ret == 0 else None
    
    def seccion_circular(self, nombre, material, diametro):
        ret = self.SapModel.PropFrame.SetCircle(nombre, material, diametro)
        return nombre if ret == 0 else None
    
    # === RESTRICCIONES ===
    def empotrar(self, punto):
        return self.SapModel.PointObj.SetRestraint(punto, [True]*6)
    
    def articular(self, punto):
        return self.SapModel.PointObj.SetRestraint(punto, [True, True, True, False, False, False])
    
    def apoyo_simple(self, punto):
        return self.SapModel.PointObj.SetRestraint(punto, [False, False, True, False, False, False])
    
    # === CARGAS ===
    def patron_carga(self, nombre, tipo='dead', factor_pp=0):
        tipos = {'dead': 1, 'superdead': 2, 'live': 3, 'quake': 5, 'wind': 6, 'other': 8}
        return self.SapModel.LoadPatterns.Add(nombre, tipos.get(tipo.lower(), 8), factor_pp)
    
    def carga_distribuida(self, frame, patron, w_kN_m, direccion='gravity'):
        dirs = {'gravity': 7, 'z': 6, 'y': 5, 'x': 4}
        return self.SapModel.FrameObj.SetLoadDistributed(
            frame, patron, 1, dirs.get(direccion.lower(), 7), 0, 1, w_kN_m, w_kN_m, "Global", True, True
        )
    
    def carga_puntual(self, frame, patron, p_kN, pos=0.5, direccion='gravity'):
        dirs = {'gravity': 7, 'z': 6, 'y': 5, 'x': 4}
        return self.SapModel.FrameObj.SetLoadPoint(
            frame, patron, 1, dirs.get(direccion.lower(), 7), pos, p_kN, "Global", True, True
        )
    
    # === ANÁLISIS ===
    def guardar(self, ruta):
        return self.SapModel.File.Save(ruta)
    
    def analizar(self):
        return self.SapModel.Analyze.RunAnalysis()

# Crear instancia
sap = SAP2000Helper(SapModel)
print("✅ Clase SAP2000Helper cargada - usa 'sap' para acceder a las funciones")

---
## 5️⃣ Ejemplo Completo - Pórtico Simple

Usando todas las funciones consolidadas para crear un modelo completo.

In [ ]:
# === EJEMPLO: Crear pórtico 2D con la clase helper ===

def crear_portico_2d(L, H, n_vanos=1, n_pisos=1, q_viga=-10, seccion_col="COL40x40", seccion_viga="VIGA30x50"):
    """
    Crea un pórtico 2D parametrizado.
    
    Parámetros:
        L: luz entre columnas (m)
        H: altura de piso (m)
        n_vanos: número de vanos
        n_pisos: número de pisos
        q_viga: carga distribuida en vigas (kN/m)
        seccion_col: nombre sección columnas
        seccion_viga: nombre sección vigas
    """
    print("=" * 50)
    print(f"Creando pórtico: {n_vanos} vanos x {n_pisos} pisos")
    print(f"Luz: {L}m, Altura: {H}m")
    print("=" * 50)
    
    # 1. Limpiar e inicializar
    sap.limpiar()
    sap.set_unidades('kN_m')
    
    # 2. Materiales
    sap.material_concreto("H30", 25e6)
    print("✅ Material H30 creado")
    
    # 3. Secciones
    sap.seccion_rectangular(seccion_col, "H30", 0.4, 0.4)
    sap.seccion_rectangular(seccion_viga, "H30", 0.5, 0.3)
    print(f"✅ Secciones creadas: {seccion_col}, {seccion_viga}")
    
    # 4. Crear geometría
    columnas = []
    vigas = []
    
    for piso in range(n_pisos):
        z_base = piso * H
        z_top = (piso + 1) * H
        
        for vano in range(n_vanos + 1):
            x = vano * L
            
            # Columna
            col = sap.frame_por_coord(x, 0, z_base, x, 0, z_top, seccion_col, f"C{piso+1}_{vano+1}")
            columnas.append(col)
            
            # Empotrar base
            if piso == 0:
                pi, pj = sap.puntos_de_frame(col)
                sap.empotrar(pi)
        
        # Vigas del piso
        for vano in range(n_vanos):
            x_i = vano * L
            x_j = (vano + 1) * L
            viga = sap.frame_por_coord(x_i, 0, z_top, x_j, 0, z_top, seccion_viga, f"V{piso+1}_{vano+1}")
            vigas.append(viga)
    
    print(f"✅ Geometría: {len(columnas)} columnas, {len(vigas)} vigas")
    
    # 5. Cargas
    for viga in vigas:
        sap.carga_distribuida(viga, "DEAD", q_viga)
    print(f"✅ Carga {q_viga} kN/m aplicada a vigas")
    
    # 6. Refrescar
    sap.refrescar()
    
    print("\n📊 Resumen:")
    print(sap.estado())
    
    return {'columnas': columnas, 'vigas': vigas}

# Ejecutar
resultado = crear_portico_2d(L=5, H=3, n_vanos=2, n_pisos=3, q_viga=-15)

In [ ]:
# === Guardar y analizar ===
import os

ruta = os.path.join(os.getcwd(), "portico_ejemplo.sdb")
sap.guardar(ruta)
print(f"💾 Guardado en: {ruta}")

ret = sap.analizar()
if ret == 0:
    print("✅ Análisis completado!")

---
## 6️⃣ Área de Experimentación

Usa estas celdas vacías para probar nuevas funciones de la API.

In [ ]:
# === Tu prueba aquí ===
# Recuerda:
# 1. ret = SapModel.XXX.YYY(params, 0, [], "")  <- valores dummy para ByRef
# 2. if ret[-1] == 0:  <- verificar éxito
# 3. resultado = ret[0]  <- extraer valor


In [ ]:
# === Otra prueba ===


---
## 📝 Notas y Observaciones

Usa esta sección para documentar lo que descubras:

- 
- 
-